# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weathercsv = "../output_data/weatherdata.csv"
weather_df = pd.read_csv(weathercsv, encoding="latin1")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Coquimbo,-29.9533,-71.3436,51.53,87,0,3.44,CL,1627773582
1,Saskylakh,71.9167,114.0833,73.00,38,0,8.75,RU,1627772996
2,Hofn,64.2539,-15.2082,51.01,90,21,7.43,IS,1627772991
3,Codrington,-38.2667,141.9667,44.56,88,27,8.14,AU,1627773429
4,VicuÃ±a,-30.0319,-70.7081,44.33,31,60,1.57,CL,1627773583


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
location = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations=location, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_1=weather_df.loc[(weather_df['Max Temp']>70)&(weather_df['Max Temp']<=80)]
weather_2=weather_1.loc[(weather_1['Wind Speed']<10)]
hotel_df=weather_2.loc[(weather_2['Cloudiness']==0)]
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,Saskylakh,71.9167,114.0833,73.00,38,0,8.75,RU,1627772996
57,Bloomsburg,41.0037,-76.4549,77.72,66,0,2.53,US,1627773398
88,Nelson Bay,-32.7167,152.1500,75.76,72,0,6.82,AU,1627773615
151,Mentougou,39.9392,116.0917,79.74,67,0,6.22,CN,1627773634
154,Mitsamiouli,-11.3847,43.2844,76.68,69,0,8.14,KM,1627773054


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""
hotel_df.head()


<ipython-input-6-64598bec773a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,Saskylakh,71.9167,114.0833,73.00,38,0,8.75,RU,1627772996,
57,Bloomsburg,41.0037,-76.4549,77.72,66,0,2.53,US,1627773398,
88,Nelson Bay,-32.7167,152.1500,75.76,72,0,6.82,AU,1627773615,
151,Mentougou,39.9392,116.0917,79.74,67,0,6.22,CN,1627773634,
154,Mitsamiouli,-11.3847,43.2844,76.68,69,0,8.14,KM,1627773054,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": "50000",
    "type": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get latitute and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel to {city_name} is {results[0]['name']}.")
    
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

Retrieving Results for Index 1: Saskylakh.
Closest hotel to Saskylakh is Saskylakh.
Retrieving Results for Index 57: Bloomsburg.


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel to Bloomsburg is Hazleton.
Retrieving Results for Index 88: Nelson Bay.
Closest hotel to Nelson Bay is Newcastle.
Retrieving Results for Index 151: Mentougou.
Closest hotel to Mentougou is Beijing.
Retrieving Results for Index 154: Mitsamiouli.
Closest hotel to Mitsamiouli is Moroni.
Retrieving Results for Index 160: Grand Gaube.
Closest hotel to Grand Gaube is Port Louis.
Retrieving Results for Index 224: Taiyuan.
Closest hotel to Taiyuan is Taiyuan.
Retrieving Results for Index 252: Porto Velho.
Closest hotel to Porto Velho is Porto Velho.
Retrieving Results for Index 254: Charters Towers.
Closest hotel to Charters Towers is Charters Towers.
Retrieving Results for Index 313: Provincia di Campobasso.
Closest hotel to Provincia di Campobasso is Campobasso.
Retrieving Results for Index 376: Fasa.
Closest hotel to Fasa is Jahrom.
Retrieving Results for Index 434: Neijiang.
Closest hotel to Neijiang is Chongqing.
Retrieving Results for Index 440: Bashtanka.
Closest hotel to 

In [8]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,Saskylakh,71.9167,114.0833,73.00,38,0,8.75,RU,1627772996,Saskylakh
57,Bloomsburg,41.0037,-76.4549,77.72,66,0,2.53,US,1627773398,Hazleton
88,Nelson Bay,-32.7167,152.1500,75.76,72,0,6.82,AU,1627773615,Newcastle
151,Mentougou,39.9392,116.0917,79.74,67,0,6.22,CN,1627773634,Beijing
154,Mitsamiouli,-11.3847,43.2844,76.68,69,0,8.14,KM,1627773054,Moroni


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))